In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

%matplotlib inline

### 0. Загрузка и первичная обработка данных

In [2]:
#Информация об оценках в тесте
scores = pd.read_csv('assessments.csv')
scores = scores.rename(columns={'code_presentation':'semestr',
                                'code_module':'course',
                                'id_assessment':'id_test',
                                'assessment_type':'check_type'})
#Список предметов по семестрам
courses = pd.read_csv('courses.csv')
courses = courses.rename(columns={'code_presentation':'semestr',
                                'code_module':'course',
                                'module_presentation_length':'semestr_day'})
#Результаты тестов студентов
student_scores = pd.read_csv('studentAssessment.csv')
student_scores = student_scores.rename(columns={'id_assessment':'id_test',
                                'date_submitted':'test_date'})
#Таблица регистрации студентов на курсы
student_start = pd.read_csv('studentRegistration.csv')
student_start = student_start.rename(columns={'code_presentation':'semestr',
                                'code_module':'course',
                                'date_registration':'date_reg',
                                'date_unregistration':'date_unreg'})

### 1. Сколько студентов успешно сдали только один курс? 

(Успешная сдача — это зачёт по курсу на экзамене - согласно таблице результатов тестов студентов порог 40 баллов)

In [3]:
#Отбор только тестов типа "Экзамен"
exams = scores.query('check_type =="Exam"')[['id_test']]

#Присоединим к полученным данным результаты успеваемости студентов, которые прошли порог в 40 баллов
student_exams = exams.merge(student_scores, how='inner', on='id_test').query('score >= 40')

#Создадим таблицу с данными о том, сколько курсов сдали студенты всего
student_exams_count = student_exams.groupby('id_student').agg({'id_test':'count'})

#Отберем из получившейся таблицы только тех, кто сдал всего 1 курс и посчитаем их количество
result = student_exams_count.query('id_test == 1').count()
print(result[0], 'студента сдали только 1 экзамен')

3802 студента сдали только 1 экзамен


### 2. Выявить самый сложный и самый простой курс\экзамен

(завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен)

In [4]:
#Подготовка (объединение) общих данных по оценкам в тесте и успеваемости студентов
scores_students_data = scores.merge(student_scores, how='inner',on='id_test')

#Выявление количества попыток сдачи тестов
attempt_total = scores_students_data.groupby(['course', 'check_type','id_test'], as_index=False)\
    .agg({'semestr':'count'}).rename(columns={'semestr':'total_attempt'})
    
#Выявление количества успешных попыток сдачи тестов(преодолевшик 40-балльный порог)
attempt_success = scores_students_data.query('score >= 40')\
    .groupby(['course', 'check_type','id_test'], as_index=False)\
    .agg({'semestr':'count'}).rename(columns={'semestr':'success_attempt'})
    
#Объединение полученных данных
result_data = attempt_total.merge(attempt_success, how='inner', on=['course', 'check_type', 'id_test'])

#Расчет уровня завершаемости
result_data['close_course'] = round(result_data.success_attempt / result_data.total_attempt *100, 2)

#Получение результатов
hard = result_data.nsmallest(1, 'close_course', keep='all')['id_test'].to_list()
easy = result_data.nlargest(1, 'close_course', keep='all')['id_test'].to_list()

print('Самые сложные курсы (ID):', *hard, '\nСамые легкие курсы (ID):', *easy)

Самые сложные курсы (ID): 15020 
Самые легкие курсы (ID): 34879 34892 34896 34907 37429


### 3. Определить по каждому предмету средний срок сдачи экзаменов

(под сдачей понимаем последнее успешное прохождение экзамена студентом)

In [5]:
#Общие данные по оценкам в тесте и успеваемости студентов были рассчитаны в разделе 2
scores_students_data

#Отбираем успешные сдачи
success = scores_students_data.query('score >= 40 and is_banked == 0')

#Выявляем для каждого студента максимальную дату сдачи курса
success_max = success.groupby(['id_student','course'],as_index=False).agg({'test_date':'max'})

#Определяем среднее количество дней для сдачи курса студентом
avg_days = success_max.groupby('course', as_index=False).agg({'test_date':'mean'})\
    .rename(columns={'test_date':'mean_days'})
avg_days['mean_days'] = avg_days.mean_days.apply(lambda x: round(x,0))

print(avg_days)

  course  mean_days
0    AAA      193.0
1    BBB      166.0
2    CCC      172.0
3    DDD      179.0
4    EEE      138.0
5    FFF      169.0
6    GGG      195.0


### 4. Выявить самые популярные предметы по количеству регистраций, 
### а также предметы с самым большим оттоком (ТОП-3)

In [6]:
#Необходимые для решения данные содержатся в таблице регистрации студентов на курсы. 
# Определим какое количество студентов было записано на каждый курс
courses_pop = student_start.groupby('course', as_index=False)\
    .agg({'id_student':'nunique'})

#Отберем топ-3 популярных предмета по количеству регистраций
top_popular = courses_pop.rename(columns={"id_student":"student_count"})\
    .sort_values('student_count', ascending=False).head(3)

# Определим какое количество студентов аннулировали свои записи на каждом курсе
courses_out = student_start[~student_start.isnull().any(axis=1)].groupby('course', as_index=False)\
    .agg({'id_student':'nunique'})

#Отберем топ-3 предмета с самым большим оттоком
top_out = courses_out.rename(columns={"id_student":"student_count"})\
    .sort_values('student_count', ascending=False).head(3)
    
print('Самые популярные курсы по кол-ву регистраций:', *top_popular['course'].to_list(), '\nКурсы с самым большим оттоком:', *top_out['course'].to_list())

Самые популярные курсы по кол-ву регистраций: BBB FFF DDD 
Курсы с самым большим оттоком: BBB FFF DDD


### 5. Выявить семестры с самой низкой звершаемостью курсов, 
### а также самыми долгими средними сроками сдачи в период с начала 2013 по конец 2014

(завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен)

In [7]:
#Общие данные по курсам и успеваемости студентов были рассчитаны в разделе 2
scores_students_data

#Находим количество студентов, взявших курс в семестре
total_students = scores_students_data.query('is_banked == 0')\
    .groupby('semestr', as_index=False)\
    .agg({'id_student':'count'}).rename(columns={'id_student':'students_total'})

#Находим количество студентов, успешно прошедших курс в каждом семестре
success_students = scores_students_data.query('is_banked == 0 and score>=40') \
                        .groupby(['semestr'], as_index=False) \
                        .agg({'id_student':'count'}) \
                        .rename(columns={'id_student':'students_success'})
                        
#Формируем итоговую таблицу на основании полученных данных
result_final = total_students.merge(success_students, how = 'inner', on = 'semestr')

#Вычисляем процент завершаемости курсов по семестрам
result_final['close_percent'] = round(result_final.students_success / result_final.students_total *100, 0)

print('Семестр с самой низкой завершаемостью курсов:', *result_final.nsmallest(1, 'close_percent', keep = 'all')['semestr'].to_list())

#Вычисляем средние сроки сдачи курсов по семестрам
long_courses = scores_students_data.query('is_banked == 0 and score>=40') \
                        .groupby('semestr', as_index=False) \
                        .agg({'test_date':'mean'})\
                        .sort_values('test_date', ascending=False)

print('Семестр с самыми долгими средними сроками сдачи курсов:', *long_courses.nlargest(1, 'test_date', keep = 'all')['semestr'].to_list())

Семестр с самой низкой завершаемостью курсов: 2014J
Семестр с самыми долгими средними сроками сдачи курсов: 2013J


### 6. Построение RFM - кластеров студентов для оценки аудитории

R - среднее время сдачи одного курса, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен

In [8]:
#Рассчет основных показателей для кластеризации
# Расчет среднего времени сдачи одного курса (recency)
recency = scores_students_data.query('is_banked == 0 and score >= 40') \
                    .groupby('id_student', as_index = False) \
                    .agg({'test_date':'mean'}) \
                    .rename(columns={'test_date' : 'Recency'})\
                    
# Расчет звершаемости курсов (frequency)
frequency = scores_students_data.query('is_banked == 0 and score >= 40') \
                    .groupby('id_student', as_index = False) \
                    .agg({'id_test':'count'}) \
                    .rename(columns = {'id_test':'count_success'}) \
.merge(scores_students_data.query('is_banked == 0') \
                    .groupby('id_student', as_index = False) \
                    .agg({'id_test':'count'}) \
                    .rename(columns = {'id_test':'count_total'}),
     how = 'inner', on ='id_student') 
frequency = frequency.assign( Frequency = frequency.count_success/frequency.count_total*100)

# Расчет среднего количества баллов за тест\экзамен (monetary)
monetary = scores_students_data.query('is_banked == 0') \
                    .groupby('id_student', as_index = False) \
                    .agg({'score':'mean'}) \
                    .rename(columns = {'score':'Monetary'})
                    
#Объединение полученных данных в таблицу
rfm = recency.merge(frequency, how = 'inner', on = 'id_student') \
               .merge(monetary, how = 'inner', on = 'id_student') \
               [['id_student', 'Recency', 'Frequency', 'Monetary']]
               
rfm = np.round(rfm, 0)
rfm

,id_student,Recency,Frequency,Monetary
0,6516,112.0,100.0,62.0
1,8462,55.0,100.0,88.0
2,11391,112.0,100.0,82.0
3,23629,56.0,100.0,82.0
4,23698,133.0,100.0,74.0
...,...,...,...,...
22908,2698251,108.0,86.0,58.0
22909,2698257,116.0,100.0,68.0
22910,2698535,73.0,50.0,39.0
22911,2698577,106.0,100.0,64.0


In [9]:
#Разбиваем данные на диапазоны. Для начального понимания аудитории 
# используем разбивку по 2 группы на каждый показатель
quantiles = rfm[['Recency', 'Frequency', 'Monetary']].quantile(0.5).to_dict()
quantiles

{'Recency': 110.0, 'Frequency': 100.0, 'Monetary': 76.0}

In [10]:
rfm.Frequency.value_counts()

Frequency
100.0    18353
80.0       739
50.0       377
86.0       369
67.0       368
         ...  
46.0         1
76.0         1
36.0         1
68.0         1
74.0         1
Name: count, Length: 62, dtype: int64

Ввиду неприменимости среднего к показателю Frequency из-за 
абсолютного большинства студентов со 100% завершаемостью курсов, 
ранги по ней будут следующие: 

1(True) - всё сдано на 100%; 

0(False) - часть курсов не сдана. 

Остальные метрики будут рассчитываться как 

0(False) - меньше среднего значения; 

1(True) - больше среднего значения.

In [11]:
# Присвоим ранги для Recency (0 лучше чем 1)
rfm['R'] = rfm.Recency.apply(lambda x: x > quantiles['Recency'])

# Присвоим ранги для Frequency (1 лучше чем 0)
rfm['F'] = rfm.Frequency.apply(lambda x: x == quantiles['Frequency'])

# Присвоим ранги для Monetary (1 лучше чем 0)
rfm['M'] = rfm.Monetary.apply(lambda x: x == quantiles['Monetary'])

rfm.head()

,id_student,Recency,Frequency,Monetary,R,F,M
0,6516,112.0,100.0,62.0,True,True,False
1,8462,55.0,100.0,88.0,False,True,False
2,11391,112.0,100.0,82.0,True,True,False
3,23629,56.0,100.0,82.0,False,True,False
4,23698,133.0,100.0,74.0,True,True,False


Далее необходимо разбить студентов по сегментам, в зависимости от полученных рангов. В качестве самого удобного решения для данной задачи будет создан отдельный столбец с текстовым описанием ранга, которое будет присваиваться в зависимости от комбинации оценок.

In [12]:
#Создаем столбец с рангами
rfm['Rang'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)

#Создаем словарь с текстовыми описаниями рангов для комбинаций оценок
rang_map = {
    r'FalseFalseFalse': 'Потенциальный отстающий', 
    r'FalseFalseTrue': 'Хорошист', 
    r'FalseTrueFalse': 'Потенциальный хорошист', 
    r'FalseTrueTrue': 'Отличник учебы', 
    r'TrueFalseFalse': 'Сильно отстающий', 
    r'TrueFalseTrue': 'Отстающий', 
    r'TrueTrueFalse': 'Удовлетворительно', 
    r'TrueTrueTrue': 'Потенциальный отличник учебы'} 
rfm

#Заменяем комбинации оценок на описания рангов
rfm['Rang'] = rfm['Rang'].replace(rang_map, regex=True)
rfm.head()

,id_student,Recency,Frequency,Monetary,R,F,M,Rang
0,6516,112.0,100.0,62.0,True,True,False,Удовлетворительно
1,8462,55.0,100.0,88.0,False,True,False,Потенциальный хорошист
2,11391,112.0,100.0,82.0,True,True,False,Удовлетворительно
3,23629,56.0,100.0,82.0,False,True,False,Потенциальный хорошист
4,23698,133.0,100.0,74.0,True,True,False,Удовлетворительно


In [13]:
# посчитаем кол-во студентов в получившихся сегментах и построим диаграмму

segment_by = rfm.groupby('Rang', as_index=False)  \
                .agg({'id_student':'count'}) \
                .rename(columns={'id_student':'count_student'}) \
                .sort_values('count_student', ascending=False)
                
#Создадим колонку с разбивкой по процентам от общего числа студентов
segment_by['Percent'] = round(segment_by.count_student / sum(segment_by.count_student)*100, 1)

segment_by

,Rang,count_student,Percent
6,Удовлетворительно,9020,39.4
4,Потенциальный хорошист,8671,37.8
3,Потенциальный отстающий,2582,11.3
5,Сильно отстающий,1923,8.4
2,Потенциальный отличник учебы,341,1.5
0,Отличник учебы,321,1.4
1,Отстающий,30,0.1
7,Хорошист,25,0.1


In [14]:
#Построим график
fig = px.treemap(segment_by, path=['Rang'], values='count_student')
fig.update_layout(title="Tree map of Segments", width=700, height=500,)
fig.show()